In [91]:
import pandas as pd
import numpy as np
import plotly.io as pio

# pio.renderers.default = 'png'

repo1 = r"C:/Users/benno/OneDrive/Data/Kaggle/weather/jackson hole - wyoming.csv"
repo2 = r"C:/Users/benno/OneDrive/Data/Kaggle/weather/Snowbird - utah.csv"
repo3 = r"C:/Users/benno/OneDrive/Data/Kaggle/weather/telluride - colorado.csv"
repo4 = r"C:/Users/benno/OneDrive/Data/Kaggle/weather/whistler blackcomb - bc canada.csv"

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment',None)
np.set_printoptions(threshold=np.inf)

jh = pd.read_csv(repo1, parse_dates=["Date"])
sb = pd.read_csv(repo2, parse_dates=["Date"])
tu = pd.read_csv(repo3, parse_dates=["Date"])
wb = pd.read_csv(repo4, parse_dates=["Date"])

In [92]:
import warnings
warnings.filterwarnings("ignore")

def summary_dataframe_small(df):

	return pd.DataFrame({'features': df.columns,'dtypes': [df.dtypes[i] for i in range(len(df.columns))],
	'nulls':[df[df.columns[i]].isnull().sum() for i in range(len(df.columns))],'nonnuls / 1.0': (df.shape[0] - 
	df.isnull().sum().values) / df.shape[0],'uniques': [df[j].nunique() for j in df.columns]})

summary_dataframe_small(sb)

,features,dtypes,nulls,nonnuls / 1.0,uniques
0,Date,datetime64[ns],0,1.0,586
1,24 hr New Snow,object,0,1.0,25
2,Season Snowfall Total,object,0,1.0,467
3,Base Depth,object,0,1.0,123


In [93]:
accum["Date"] = pd.to_datetime(accum["Date"])

In [94]:
d1 = ( wb["Date"] >= "2012-1-1" )
d2 = ( wb["Date"] < "2012-2-1" )
s2011 = ( wb["Base Depth"] =="102 cm")
wb[d1 & d2 & s2011]
idxs = [370, 371, 372, 375, 276, 377, 378, 379, 380]
wb.loc[idxs, "Date"] = wb.loc[idxs, "Date"] - timedelta(365)

In [95]:
import plotly.express as px

In [96]:
# Jackson Hole
# 43.4799 N
# 110.7624 W,
jh["Latitude"] = 43.4799
jh["Longitude"] =  -110.7624

In [97]:
# Snowbird
# 40.5829° N, 
# 111.6556 W
sb["Latitude"] =40.5829
sb["Longitude"] =  -111.6556

In [98]:
# Telluride
# 37.9375° N, 
# 107.8123 W,
tu["Latitude"] = 37.9375
tu["Longitude"] = -107.8123

In [99]:
# Whistler
# 50.1162° N, 
# 122.9535 W,
wb["Latitude"] = 50.1162
wb["Longitude"] = -122.9535

In [100]:
jh["Resort"] = "jackson hole"
sb["Resort"] = "snowbird"
tu["Resort"] = "telluride"
wb["Resort"] = "whistler blackcomb"

In [101]:
resorts = pd.concat([jh, sb, tu, wb])

In [102]:
resorts.columns = ["Date", "LastDay", "Total", "BaseDepth", "Latitude", "Longitude", "Resort"]

In [104]:
resorts.dtypes

Date         datetime64[ns]
LastDay              object
Total                object
BaseDepth            object
Latitude            float64
Longitude           float64
Resort               object
dtype: object

In [105]:
resorts["LastDay"] = [int(j.split(" ")[0]) for j in resorts["LastDay"]]

In [106]:
resorts["Total"] = [int(j.split(" ")[0]) for j in resorts["Total"]]

In [107]:
resorts["BaseDepth"] = [int(j.split(" ")[0]) for j in resorts["BaseDepth"]]

In [108]:
from datetime import datetime

def extract_time(df1, timecolumn):
    for df in [df1]:
        df['Datetime'] = df[timecolumn].apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d %H:%M:%S"))
        df['Weekday'] = [t.day_name() for t in df.Datetime]
        df['Month'] = [t.month_name() for t in df.Datetime]
        df['Day'] = [t.day for t in df.Datetime]
        df['Hour'] = [t.hour for t in df.Datetime]
        df['Minute'] = [t.minute for t in df.Datetime]
        df['Date'] = [t.date() for t in df.Datetime]
        
    return df

resorts = extract_time(resorts,"Date")

In [109]:
resorts = resorts.sort_values(by="Datetime", ascending=True)
resorts = resorts.reset_index(drop=True)

In [110]:
resorts[:5]

,Date,LastDay,Total,BaseDepth,Latitude,Longitude,Resort,Datetime,Weekday,Month,Day,Hour,Minute
0,2009-01-01,18,18,142,43.4799,-110.7624,jackson hole,2009-01-01,Thursday,January,1,0,0
1,2009-01-01,10,10,124,50.1162,-122.9535,whistler blackcomb,2009-01-01,Thursday,January,1,0,0
2,2009-01-02,23,41,147,43.4799,-110.7624,jackson hole,2009-01-02,Friday,January,2,0,0
3,2009-01-03,30,30,178,40.5829,-111.6556,snowbird,2009-01-03,Saturday,January,3,0,0
4,2009-01-03,25,66,157,43.4799,-110.7624,jackson hole,2009-01-03,Saturday,January,3,0,0


In [111]:
resorts["Resort"].unique()

array(['jackson hole', 'whistler blackcomb', 'snowbird', 'telluride'],
      dtype=object)

In [112]:
# subtract total every august

sb = ( resorts["Resort"] == "snowbird" )
jh = ( resorts["Resort"] == "jackson hole" )
tu = ( resorts["Resort"] == "telluride" )
bl = ( resorts["Resort"] == "whistler blackcomb" )


sm1 = ( resorts["Datetime"] > "2009-08-01" )
sm2 = ( resorts["Datetime"] < "2010-08-01" )

r0910_sb = resorts[sb][sm1 & sm2]
r0910_jh = resorts[jh][sm1 & sm2]
r0910_tu = resorts[tu][sm1 & sm2]
r0910_bl = resorts[bl][sm1 & sm2]


sm1 = ( resorts["Datetime"] > "2010-08-01" )
sm2 = ( resorts["Datetime"] < "2011-08-01" )

r1011_sb = resorts[sb][sm1 & sm2]
r1011_jh = resorts[jh][sm1 & sm2]
r1011_tu = resorts[tu][sm1 & sm2]
r1011_bl = resorts[bl][sm1 & sm2]


sm1 = ( resorts["Datetime"] > "2011-08-01" )
sm2 = ( resorts["Datetime"] < "2012-08-01" )

r1112_sb = resorts[sb][sm1 & sm2]
r1112_jh = resorts[jh][sm1 & sm2]
r1112_tu = resorts[tu][sm1 & sm2]
r1112_bl = resorts[bl][sm1 & sm2]


sm1 = ( resorts["Datetime"] > "2012-08-01" )
sm2 = ( resorts["Datetime"] < "2013-08-01" )

r1213_sb = resorts[sb][sm1 & sm2]
r1213_jh = resorts[jh][sm1 & sm2]
r1213_tu = resorts[tu][sm1 & sm2]
r1213_bl = resorts[bl][sm1 & sm2]


sm1 = ( resorts["Datetime"] > "2013-08-01" )
sm2 = ( resorts["Datetime"] < "2014-08-01" )

r1314_sb = resorts[sb][sm1 & sm2]
r1314_jh = resorts[jh][sm1 & sm2]
r1314_tu = resorts[tu][sm1 & sm2]
r1314_bl = resorts[bl][sm1 & sm2]


sm1 = ( resorts["Datetime"] > "2014-08-01" )
sm2 = ( resorts["Datetime"] < "2015-08-01" )

r1415_sb = resorts[sb][sm1 & sm2]
r1415_jh = resorts[jh][sm1 & sm2]
r1415_tu = resorts[tu][sm1 & sm2]
r1415_bl = resorts[bl][sm1 & sm2]


sm1 = ( resorts["Datetime"] > "2015-08-01" )
sm2 = ( resorts["Datetime"] < "2016-08-01" )

r1516_sb = resorts[sb][sm1 & sm2]
r1516_jh = resorts[jh][sm1 & sm2]
r1516_tu = resorts[tu][sm1 & sm2]
r1516_bl = resorts[bl][sm1 & sm2]


sm1 = ( resorts["Datetime"] > "2016-08-01" )
sm2 = ( resorts["Datetime"] < "2017-08-01" )

r1617_sb = resorts[sb][sm1 & sm2]
r1617_jh = resorts[jh][sm1 & sm2]
r1617_tu = resorts[tu][sm1 & sm2]
r1617_bl = resorts[bl][sm1 & sm2]

In [113]:
def rolsum(df, column):
    return np.cumsum(df[column])

In [114]:
r0910_sb["SeasonalSum"] = rolsum(r0910_sb, "LastDay")
r0910_jh["SeasonalSum"] = rolsum(r0910_jh, "LastDay")
r0910_tu["SeasonalSum"] = rolsum(r0910_tu, "LastDay")
r0910_bl["SeasonalSum"] = rolsum(r0910_bl, "LastDay")

r1011_sb["SeasonalSum"] = rolsum(r1011_sb, "LastDay")
r1011_jh["SeasonalSum"] = rolsum(r1011_jh, "LastDay")
r1011_tu["SeasonalSum"] = rolsum(r1011_tu, "LastDay")
r1011_bl["SeasonalSum"] = rolsum(r1011_bl, "LastDay")

r1112_sb["SeasonalSum"] = rolsum(r1112_sb, "LastDay")
r1112_jh["SeasonalSum"] = rolsum(r1112_jh, "LastDay")
r1112_tu["SeasonalSum"] = rolsum(r1112_tu, "LastDay")
r1112_bl["SeasonalSum"] = rolsum(r1112_bl, "LastDay")

r1213_sb["SeasonalSum"] = rolsum(r1213_sb, "LastDay")
r1213_jh["SeasonalSum"] = rolsum(r1213_jh, "LastDay")
r1213_tu["SeasonalSum"] = rolsum(r1213_tu, "LastDay")
r1213_bl["SeasonalSum"] = rolsum(r1213_bl, "LastDay")

r1314_sb["SeasonalSum"] = rolsum(r1314_sb, "LastDay")
r1314_jh["SeasonalSum"] = rolsum(r1314_jh, "LastDay")
r1314_tu["SeasonalSum"] = rolsum(r1314_tu, "LastDay")
r1314_bl["SeasonalSum"] = rolsum(r1314_bl, "LastDay")

r1415_sb["SeasonalSum"] = rolsum(r1415_sb, "LastDay")
r1415_jh["SeasonalSum"] = rolsum(r1415_jh, "LastDay")
r1415_tu["SeasonalSum"] = rolsum(r1415_tu, "LastDay")
r1415_bl["SeasonalSum"] = rolsum(r1415_bl, "LastDay")

r1516_sb["SeasonalSum"] = rolsum(r1516_sb, "LastDay")
r1516_jh["SeasonalSum"] = rolsum(r1516_jh, "LastDay")
r1516_tu["SeasonalSum"] = rolsum(r1516_tu, "LastDay")
r1516_bl["SeasonalSum"] = rolsum(r1516_bl, "LastDay")

r1617_sb["SeasonalSum"] = rolsum(r1617_sb, "LastDay")
r1617_jh["SeasonalSum"] = rolsum(r1617_jh, "LastDay")
r1617_tu["SeasonalSum"] = rolsum(r1617_tu, "LastDay")
r1617_bl["SeasonalSum"] = rolsum(r1617_bl, "LastDay")

In [115]:
accum = pd.concat([
    r0910_sb, 
    r0910_jh, 
    r0910_tu, 
    r0910_bl, 

    r1011_sb, 
    r1011_jh, 
    r1011_tu, 
    r1011_bl, 

    r1112_sb, 
    r1112_jh, 
    r1112_tu, 
    r1112_bl, 

    r1213_sb, 
    r1213_jh, 
    r1213_tu, 
    r1213_bl, 

    r1314_sb, 
    r1314_jh, 
    r1314_tu, 
    r1314_bl, 

    r1415_sb, 
    r1415_jh, 
    r1415_tu, 
    r1415_bl, 

    r1516_sb, 
    r1516_jh, 
    r1516_tu, 
    r1516_bl, 

    r1617_sb, 
    r1617_jh, 
    r1617_tu, 
    r1617_bl, 
    
    ])

In [116]:
blues = px.colors.sequential.Blues[4:8]
dict_colors = dict(zip(accum["Resort"].unique(), blues))
accum["Colors"] = accum["Resort"].map(dict_colors)
accum = accum.reset_index(drop=True)

In [117]:
accum = accum.sort_values(by="Resort", ascending=True)

In [118]:
repo = r"C:/Users/benno/OneDrive/Python/Dash/Deployment_ready/snowfall/assets/data/accumulation.csv"

accum.to_csv(repo, index=False)

In [119]:
from plotly.offline import *
import plotly.graph_objs as go
import plotly.io as pio
from plotly.subplots import make_subplots
from itertools import *

pio.renderers.default = 'iframe'

#done

def fig_b_subplots(df1, date1, date2):

    ymask1 = ( df1["Datetime"] >= date1 )
    ymask2 = ( df1["Datetime"] <= date2 )

    accum = df1[ymask1 & ymask2]

    mask_place = ( accum["Resort"]=="jackson hole" )
    df = accum[mask_place]
    trace1 = go.Scatter(x=df["Date"], y=df["SeasonalSum"], mode="markers", marker={"color":df["Colors"], "symbol": "diamond"}, name=df["Resort"].values[0])
    
    mask_place = ( accum["Resort"]=="snowbird" )
    df = accum[mask_place]
    trace0 = go.Scatter(x=df["Date"], y=df["SeasonalSum"], mode="markers", marker={"color":df["Colors"], "symbol": "diamond"}, name=df["Resort"].values[0])
        
    mask_place = ( accum["Resort"]=="telluride" )
    df = accum[mask_place]
    trace2 = go.Scatter(x=df["Date"], y=df["SeasonalSum"], mode="markers", marker={"color":df["Colors"], "symbol": "diamond"}, name=df["Resort"].values[0])
    
    mask_place = ( accum["Resort"]=="whistler blackcomb" )
    df = accum[mask_place]
    trace3 = go.Scatter(x=df["Date"], y=df["SeasonalSum"], mode="markers", marker={"color":df["Colors"], "symbol": "diamond"}, name=df["Resort"].values[0])
    
    plots = make_subplots(rows=4, cols=1, shared_xaxes=True)
    
    plots = plots.add_trace(trace0, row=1, col=1)
    plots = plots.add_trace(trace1, row=2, col=1)
    plots = plots.add_trace(trace2, row=3, col=1)
    plots = plots.add_trace(trace3, row=4, col=1)
    plots = plots.update_yaxes({"range":[-400, 1900]}).update_layout({
        "legend": {"orientation": "h", "y": 1.1}, 
        "title": "Snowfall, across resorts, across time",
    })
    
    return plots
fig_b_subplots(accum, "2009-10-01", "2018-04-08")

In [ ]:
whist = ( accum["Resort"] == "whistler blackcomb" )

In [ ]:
whist = ( accum["Resort"] == "whistler blackcomb" )
d1 = ( accum["Datetime"] >= "2012-01-01" )
d2 = ( accum["Datetime"] < "2012-02-01" )
s2011 = ( accum["BaseDepth"] < 150 )

In [ ]:
s2011 = ( accum["BaseDepth"] < 150 )

In [ ]:
accum.loc[idxs, "Date"] = accum.loc[idxs, "Date"] - timedelta(365)

In [ ]:
whist = ( accum["Resort"] == "whistler blackcomb" )
d1 = ( accum["Datetime"] >= "2012-01-01" )
d2 = ( accum["Datetime"] < "2012-02-01" )
s2011 = ( accum["BaseDepth"] < 150 )
accum.loc[idxs, "Date"]
# accum.loc[idxs, "Date"] = accum.loc[idxs, "Date"] - timedelta(365)
print(len(accum[whist & d1 & d2].drop_duplicates(subset="Date")))
print(len(accum[whist & d1 & d2]))
accum[whist & d1 & d2 & s2011].sort_values(by=["Date"], ascending=True)
# 
idxs = [732, 733, 734, 737, 738, 739, 740, 741, 742]
accum.loc[idxs]

In [ ]:
print(len(accum[whist & d1 & d2].drop_duplicates(subset="Date")))
print(len(accum[whist & d1 & d2]))
accum[whist & d1 & d2 & s2011].sort_values(by=["Date"], ascending=True)
# 
idxs = [732, 733, 734, 737, 738, 739, 740, 741, 742]
accum.loc[idxs]

In [ ]:
from datetime import date, timedelta, datetime
accum[whist & d1 & d2 & s2011]["Date"] = accum[whist & d1 & d2 & s2011]["Date"] - timedelta(365)

In [ ]:
from datetime import date, timedelta, datetime

# date1 = pd.to_datetime("2008-08-01")
date2 = pd.to_datetime("2017-08-01")

def fig_g_messyscatter(df, date2):
    
    season1 = datetime(pd.to_datetime(date2).year, 8, 1) - timedelta(365)
    season2 = datetime(pd.to_datetime(date2).year, 8, 1)
    

    smask1 = ( df["Datetime"] >= str(season1) )
    smask2 = ( df["Datetime"] <= str(season2) )
    df = df[smask1 & smask2]        
    fig = px.scatter(x=df["Datetime"], y=df["SeasonalSum"], color=df["Resort"], symbol_sequence=["diamond"], color_discrete_sequence=px.colors.sequential.Blues[4:])

    return fig.update_layout({
        "title": "Snowfall across time, most recent season",
        "xaxis": {"title": "Date"},
        "yaxis": {"title": "Aggregate snowfall (cm)"},
        "legend": {"title": "Resort"}
    })

fig_g_messyscatter(accum, date2)

In [ ]:
# done

def fig_e_balls(df1, date1, date2):
    
    ymask1 = ( df1["Datetime"] >= date1 )
    ymask2 = ( df1["Datetime"] <= date2 )

    gb = df1.groupby(["Resort"]).agg({"SeasonalSum": "sum", "Colors": "min"}).reset_index()
    gb["column"] = [1, 2, 1, 2]
    gb["row"] = [1, 1, 2, 2]

    cd = pd.concat([pd.Series(gb["Resort"]), pd.Series(gb["SeasonalSum"])], axis=1)
    
    trace = go.Scatter(
        x=gb["column"],
        y=gb["row"],
        mode="markers",
        marker={"size": gb["SeasonalSum"] / 5000, "color": gb["Colors"]},
        name="",
        customdata=cd,
        hovertemplate=
        "Resort: %{customdata[0]}" +
        "<br>Total Snowfall: %{customdata[1]}"
    )        

    fig = go.Figure(trace).update_layout({
        "xaxis": {"showticklabels": False, "range": [0, 3]}, 
        "yaxis": {"showticklabels": False, "range":[0, 3]}, 
        "clickmode": "select",
        "title": "Aggregate snowfall as size, full time window"
    })
    
    return fig
    
fig_e_balls(accum, "2008-10-01", "2018-04-08")

In [ ]:
# len(px.colors.sequential.Blues)

In [ ]:
# gb = resorts[mk_year].groupby(["Resort"]).agg({"Total" : "max"}).reset_index()
# gb["column"] = [1, 2, 1, 2]
# gb["row"] = [1, 1, 2, 2]
# gb

In [ ]:
# cd = pd.concat([pd.Series(gb["Resort"]), pd.Series(gb["Total"])], axis=1)

# trace = go.Scatter(
#     x=gb["column"],
#     y=gb["row"],
#     mode="markers",
#     marker={"size": gb["Total"] / 6, "color": gb["Colors"]},
#     name="",
#     customdata=cd,
#     hovertemplate=
#     "Resort: %{customdata[0]}" +
#     "<br>Total Snowfall: %{customdata[1]}"
# )
# go.Figure(trace).update_layout({"xaxis": {"range": [0, 3]}, "yaxis": {"range":[0, 3]}, "clickmode": "select"}).update_layout({"height": 600, "width": 600})

In [ ]:
# resorts["Colors"] = resorts["Resort"].map(dict_colors)

In [ ]:
# resorts[:5]

In [ ]:
def geographic():
    
    fig = px.scatter_geo(accum, lat="Latitude", lon="Longitude", scope="world", color="Resort", color_discrete_sequence=px.colors.sequential.Blues[4:])
    fig.update_geos({
        "projection" :{"type": "orthographic", 
        "distance": 1.5,
        "rotation": {"lat":40, "lon": -110}},
        "showcountries":True, 
        "countrycolor":"Black",
    }).update_layout({
        "legend": {"visible": False},
        "margin":{"r":0,"t":0,"l":0,"b":0}
    })

    return fig

geographic()

In [ ]:
# pio.renderers.default = 'iframe'

# fig = px.scatter_mapbox(resorts, lat="Latitude", lon="Longitude", hover_name="Resort", size="Total", zoom= 3)
# fig.update_layout(mapbox_style="open-street-map")
# # fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# # px.scatter(resorts, x="Latitude", y="Longitude")
# fig.update_layout({"height": 400, "width": 400})

In [ ]:
def pull_categoricals(df):
    return [j for j in df.columns if "object" in str(df[j].dtype)]

categoricals = pull_categoricals(accum)

In [ ]:
def pull_continuous(df, categoricals):
    return [j for j in df.columns if j not in categoricals]

continuous = pull_continuous(accum, categoricals)

In [ ]:
"rgb(107,174,214)",
"rgb(66,146,198)",
"rgb(33,113,181)",
"rgb(8,81,156)"

In [88]:
accum[:5]

,Date,LastDay,Total,BaseDepth,Latitude,Longitude,Resort,Datetime,Weekday,Month,Day,Hour,Minute,SeasonalSum,Colors
905,2012-12-29,3,1153,122,43.4799,-110.7624,jackson hole,2012-12-29,Saturday,December,29,0,0,429,"rgb(66,146,198)"
1454,2015-03-04,3,454,66,43.4799,-110.7624,jackson hole,2015-03-04,Wednesday,March,4,0,0,1137,"rgb(66,146,198)"
1453,2015-03-03,13,451,66,43.4799,-110.7624,jackson hole,2015-03-03,Tuesday,March,3,0,0,1134,"rgb(66,146,198)"
1452,2015-02-28,13,438,64,43.4799,-110.7624,jackson hole,2015-02-28,Saturday,February,28,0,0,1121,"rgb(66,146,198)"
1451,2015-02-25,5,425,61,43.4799,-110.7624,jackson hole,2015-02-25,Wednesday,February,25,0,0,1108,"rgb(66,146,198)"


In [89]:
# done

def fig_f_bar(df1, year1, year2):

    mask1 = ( df1["Datetime"] >= date1 )
    mask2 = ( df1["Datetime"] <= date2 )
    
    df = df1[mask1 & mask2]
    
    gb = accum.groupby(["Resort"]).agg({"Total": "sum"}).reset_index()
    gb = gb.sort_values(by="Resort", ascending=True)
    fig= px.bar(gb, x="Resort", y="Total", color="Resort", color_discrete_sequence=px.colors.sequential.Blues[4:]).update_layout({"clickmode": "select"})
    return fig.update_layout({"title": "Total snowfall, all years", "yaxis": {"title": "Snowfall (cm)"}})

fig_g_bar(accum, "2012-01-01", "2018-01-01")

NameError: name 'fig_g_bar' is not defined

In [ ]:

def fig_g_bar(df1, date1, date2):

    mask1 = ( df1["Datetime"] >= date1 )
    mask2 = ( df1["Datetime"] <= date2 )
    
    df = df1[mask1 & mask2]
    
    gb = df.groupby(["Resort"])[["SeasonalSum"]].sum()
    
    summit = gb.sum(axis=0)
    gb1 = gb / summit
    
    colors = cycle(iter(px.colors.sequential.Blues[4:]))

    gb = gb.reindex(["snowbird", "jackson hole", "telluride", "whistler blackcomb"])
    
    figures = []
    
    for h in gb.index:
        mask = ( gb.index == h )
        gb2 = gb1[mask]
        x = gb2.columns.values
        y = gb2.values.flatten()
        fig = px.bar(x=x, y=y, opacity=0.8, hover_name=[h]*len(x)).update_traces({"marker" :{"color": next(colors)}})
        figures.append(fig.update_layout({"yaxis": {"showgrid": False}}))
    
    datum = figures[0].data + figures[1].data + figures[2].data + figures[3].data
    
    fig = go.Figure(datum)
    return fig.update_layout({
        "title": "Percentage of total snow, over the full time window",
        "xaxis": {"title": "Resort"},
        "yaxis": {"title": "% ofsnow"}
    })

fig_g_bar(accum, "2013-10-01", "2018-04-08")

In [ ]:
# def weekdayhm_heatmap(df, tx1="Weekday", ty1="Month", ty2="Day", value="SeasonalSum", title="Stock Price by Time", colorscale="Plasma"):

#     gb = df.groupby([tx1, ty1, ty2]).agg({value: lambda x: np.round(np.mean(x), 2)}).reset_index().sort_values([ty1, ty2], ascending=False)
#     gb[ty2] = [v if len(str(v))>1 else "0"+str(v) for v in gb[ty2].values]
#     gb["Time"] = gb[[ty1, ty2]].astype(str).agg(":".join, axis=1)
#     gb = gb[:390]

#     # title=f"Stock Price by Time, {df[ticker].values[0]}"
#     # cd_val = np.array([np.round(k, 2) for k in gb[value]]).reshape(-1,5)

#     trace1 = go.Histogram2d(
#         x = gb[tx1],
#         y = gb["Time"],
#         z = gb[value],
#         histfunc="avg",
#         # customdata=cd_val,
#         # hovertemplate=
#         # "<b>Average: %{customdata}</b><br>" +
#         # "Time: %{y}<br>" +
#         # "Day: %{x}<br>" +
#         # "<extra></extra>",
#         colorscale=colorscale
#     )

#     fig = go.Figure(data=[trace1])

#     fig.update_layout(
#         dict(
#             font={"family": "Lato", "color": "#979797"},
#             paper_bgcolor="#111111",
#             plot_bgcolor="#1e1e1e",
#             clickmode="select",
#             title=dict(
#                 text=title,
#                 font={"size": 28.5, "color": "rgba(255, 255, 255, 155)"},
#                 x=0.5,
#             ),
#             legend=dict(
#                 bgcolor="#292929",
#                 font={"color": "#e1e1e1"},
#                 x=0.01,
#                 y=0.97
#             ),
#             xaxis=dict(
#                 categoryarray=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"],
#                 categoryorder="array",
#                 showticklabels=True,
#                 zeroline=False,
#                 showgrid=False,
#             ),
#             yaxis=dict(
#                 ticks="outside",
#                 nticks=12,
#                 showdividers=False,
#                 zeroline=False,
#                 showgrid=False,
#             )
#         )
#     )

#     return fig

# weekdayhm_heatmap(accum[(accum["Resort"]=="snowbird")])